In [4]:
import pandas as pd
import datetime
from datetime import timedelta
from tqdm import tqdm_notebook

def get_naver_fd(i,j): 
    url = j

    # # 일자 데이터를 담을 df라는 DataFrame 정의 
    df = pd.DataFrame() # 1페이지에서 1000페이지의 데이터만 가져오기 

    for page in range(1,1000): 
        pg_url = '{url}&page={page}'.format(url=url, page=page) 
        df = df.append(pd.read_html(pg_url, header=0)[0], ignore_index=True) 

    # df.dropna()를 이용해 결측값 있는 행 제거 
    df = df.dropna() 

    #컬럼명 중복 로드건 제거
    df = df[df["날짜"] != '날짜']

    # 컬럼명 정비 (기관.1~5 = > 세부 명칭대로 변경)
    if i == 0:
        df.columns = ['Date',"kospi_개인",'kospi_외국인','kospi_기관총계','kospi_금융투자','kospi_보험','kospi_투신(사모)','kospi_은행','kospi_기타금융기관','kospi_연기금등','kospi_기타법인']
    elif i == 1:
        df.columns = ['Date',"kosdaq_개인",'kosdaq_외국인','kosdaq_기관총계','kosdaq_금융투자','kosdaq_보험','kosdaq_투신(사모)','kosdaq_은행','kosdaq_기타금융기관','kosdaq_연기금등','kosdaq_기타법인']
    elif i == 2:
        df.columns = ['Date',"선물_개인",'선물_외국인','선물_기관총계','선물_금융투자','선물_보험','선물_투신(사모)','선물_은행','선물_기타금융기관','선물_연기금등','선물_기타법인']

    # 날짜 형식 수정
    df["Date"] = '20' + df["Date"].str.split('.').str[0] + '-' + df["Date"].str.split('.').str[1] + '-' + df["Date"].str.split('.').str[2]
    df = df.sort_values(["Date"])
    df = df.reset_index(drop=True)
    df = df.drop_duplicates(keep=False)


    for i in tqdm_notebook(range(1, len(df.columns[1:])+1)):
        df_1 = df.iloc[:,[0,i]]
        df_1["Date"] = pd.to_datetime(df_1["Date"])
        df_1.iloc[:,1] = df_1.iloc[:,1].astype(int)

        # 매수(+) 주간단위 집계
        df_1_1 = df_1[df_1.iloc[:,1] >= 0]
        df_1_1_2 = df_1_1.groupby(pd.Grouper(key='Date', freq='W'))[df_1_1.columns[1]].sum().reset_index()
        df_1_1_2.columns.values[1] = df_1_1_2.columns.values[1] + "_매수총계"
        df_1_1_2["Date"] = df_1_1_2["Date"] - timedelta(days=2)
        df_1_1_2.to_csv("D:\\시세모니터링\\융합분석\\DATA\\" + datetime.datetime.now().isoformat()[:10].replace("-","") + "\\"+ df_1_1_2.columns.values[1] +".csv", index=False)
        print(df_1_1_2.columns.values[1], " 완료")

        # 매도(-) 주간단위 집계
        df_1_2 = df_1[df_1.iloc[:,1] < 0]
        df_1_2_2 = df_1_2.groupby(pd.Grouper(key='Date', freq='W'))[df_1_2.columns[1]].sum().reset_index()
        df_1_2_2.columns.values[1] = df_1_2_2.columns.values[1] + "_매도총계"
        df_1_2_2["Date"] = df_1_2_2["Date"] - timedelta(days=2)
        df_1_2_2.to_csv("D:\\시세모니터링\\융합분석\\DATA\\" + datetime.datetime.now().isoformat()[:10].replace("-","") + "\\"+ df_1_2_2.columns.values[1] +".csv", index=False)
        print(df_1_1_2.columns.values[1], " 완료")
    #     df1_mg.to_csv("D:\\시세모니터링\\융합분석\\DATA\\" + datetime.datetime.now().isoformat()[:10].replace("-","") + "\\"+ "kr_bs_rate"+".csv", index=False)

        del df_1_2_2
        del df_1_1_2
        del df_1_2
        del df_1

In [5]:
import datetime
url_list = [
    'https://finance.naver.com/sise/investorDealTrendDay.nhn?bizdate=' + datetime.datetime.now().isoformat()[:10].replace("-","") + '&sosok=',
    'https://finance.naver.com/sise/investorDealTrendDay.nhn?bizdate=' + datetime.datetime.now().isoformat()[:10].replace("-","") + '&sosok=02',
    'https://finance.naver.com/sise/investorDealTrendDay.nhn?bizdate=' + datetime.datetime.now().isoformat()[:10].replace("-","") + '&sosok=03'
]

In [6]:
for i,j in enumerate(tqdm_notebook(url_list)):
    get_naver_fd(i,j) 

c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\ipykernel_launcher.py:37: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
